# Semantic Routing

RedisVL provides a `SemanticRouter` interface to utilize Redis' built-in search & aggregation in order to perform
KNN-style classification over a set of `Route` references to determine the best match.

This notebook will go over how to use Redis as a Semantic Router for your applications

## Define the Routes

Below we define 3 different routes. One for `technology`, one for `sports`, and
another for `entertainment`. Now for this example, the goal here is
surely topic "classification". But you can create routes and references for
almost anything.

Each route has a set of references that cover the "semantic surface area" of the
route. The incoming query from a user needs to be semantically similar to one or
more of the references in order to "match" on the route.

In [1]:
from redisvl.extensions.router import Route


# Define routes for the semantic router
technology = Route(
    name="technology",
    references=[
        "what are the latest advancements in AI?",
        "tell me about the newest gadgets",
        "what's trending in tech?"
    ],
    metadata={"category": "tech", "priority": 1}
)

sports = Route(
    name="sports",
    references=[
        "who won the game last night?",
        "tell me about the upcoming sports events",
        "what's the latest in the world of sports?",
        "sports",
        "basketball and football"
    ],
    metadata={"category": "sports", "priority": 2}
)

entertainment = Route(
    name="entertainment",
    references=[
        "what are the top movies right now?",
        "who won the best actor award?",
        "what's new in the entertainment industry?"
    ],
    metadata={"category": "entertainment", "priority": 3}
)


## Initialize the SemanticRouter

``SemanticRouter`` will automatically create an index within Redis upon initialization for the route references. By default, it uses the `HFTextVectorizer` to 
generate embeddings for each route reference.

In [2]:
import os
from redisvl.extensions.router import SemanticRouter
from redisvl.utils.vectorize import HFTextVectorizer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the SemanticRouter
router = SemanticRouter(
    name="topic-router",
    vectorizer=HFTextVectorizer(),
    routes=[technology, sports, entertainment],
    redis_url="redis://localhost:6379",
    overwrite=True # Blow away any other routing index with this name
)

In [3]:
router.vectorizer

HFTextVectorizer(model='sentence-transformers/all-mpnet-base-v2', dims=768)

In [4]:
# look at the index specification created for the semantic router
!rvl index info -i topic-router



Index Information:
╭──────────────┬────────────────┬──────────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes         │ Index Options   │   Indexing │
├──────────────┼────────────────┼──────────────────┼─────────────────┼────────────┤
│ topic-router │ HASH           │ ['topic-router'] │ []              │          0 │
╰──────────────┴────────────────┴──────────────────┴─────────────────┴────────────╯
Index Fields:
╭────────────┬─────────────┬────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────╮
│ Name       │ Attribute   │ Type   │ Field Option   │ Option Value   │ Field Option   │ Option Value   │ Field Option   │   Option Value │ Field Option    │ Option Value   │
├────────────┼─────────────┼────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┤
│ route_name │ route

## Simple routing

In [5]:
# Query the router with a statement
route_match = router("Can you tell me about the latest in artificial intelligence?")
route_match

RouteMatch(name='technology', distance=0.119614243507)

In [6]:
# Query the router with a statement and return a miss
route_match = router("are aliens real?")
route_match

RouteMatch(name=None, distance=None)

In [7]:
# Toggle the runtime distance threshold
route_match = router("Which basketball team will win the NBA finals?", distance_threshold=0.7)
route_match

RouteMatch(name='sports', distance=0.554210186005)

We can also route a statement to many routes and order them by distance:

In [8]:
# Perform multi-class classification with route_many() -- toggle the max_k and the distance_threshold
route_matches = router.route_many("Lebron James", distance_threshold=1.0, max_k=3)
route_matches

[RouteMatch(name='sports', distance=0.758580708504),
 RouteMatch(name='entertainment', distance=0.812423825264),
 RouteMatch(name='technology', distance=0.88423516353)]

In [9]:
# Toggle the aggregation method -- note the different distances in the result
from redisvl.extensions.router.schema import DistanceAggregationMethod

route_matches = router.route_many("Lebron James", aggregation_method=DistanceAggregationMethod.min, distance_threshold=1.0, max_k=3)
route_matches

[RouteMatch(name='sports', distance=0.663253903389),
 RouteMatch(name='entertainment', distance=0.712985396385),
 RouteMatch(name='technology', distance=0.832674384117)]

Note the different route match distances. This is because we used the `min` aggregation method instead of the default `avg` approach.

## Update the routing config

In [10]:
from redisvl.extensions.router import RoutingConfig

router.update_routing_config(
    RoutingConfig(distance_threshold=1.0, aggregation_method=DistanceAggregationMethod.min, max_k=3)
)

In [11]:
route_matches = router.route_many("Lebron James")
route_matches

[RouteMatch(name='sports', distance=0.663253903389),
 RouteMatch(name='entertainment', distance=0.712985396385),
 RouteMatch(name='technology', distance=0.832674384117)]

## Router serialization

In [12]:
router.to_dict()

{'name': 'topic-router',
 'routes': [{'name': 'technology',
   'references': ['what are the latest advancements in AI?',
    'tell me about the newest gadgets',
    "what's trending in tech?"],
   'metadata': {'category': 'tech', 'priority': '1'}},
  {'name': 'sports',
   'references': ['who won the game last night?',
    'tell me about the upcoming sports events',
    "what's the latest in the world of sports?",
    'sports',
    'basketball and football'],
   'metadata': {'category': 'sports', 'priority': '2'}},
  {'name': 'entertainment',
   'references': ['what are the top movies right now?',
    'who won the best actor award?',
    "what's new in the entertainment industry?"],
   'metadata': {'category': 'entertainment', 'priority': '3'}}],
 'vectorizer': {'type': 'hf',
  'model': 'sentence-transformers/all-mpnet-base-v2'},
 'routing_config': {'distance_threshold': 1.0,
  'max_k': 3,
  'aggregation_method': 'min'}}

In [13]:
router2 = SemanticRouter.from_dict(router.to_dict(), redis_url="redis://localhost:6379")

assert router2 == router

15:16:28 redisvl.index.index INFO   Index already exists, not overwriting.


In [14]:
router.to_yaml("router.yaml", overwrite=True)

In [16]:
router3 = SemanticRouter.from_yaml("router.yaml", redis_url="redis://localhost:6379")

assert router3 == router2 == router

15:17:42 redisvl.index.index INFO   Index already exists, not overwriting.


## Clean up the router

In [17]:
# Use clear to flush all routes from the index
router.clear()

In [18]:
# Use delete to clear the index and remove it completely
router.delete()